In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cpu)
    Python  3.9.13 (you have 3.9.18)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
import pandas as pd

# Specify the path to the CSV file in your Google Drive folder
file_path = "../finetuning_data/llama_format_training_data.csv"  # Replace with the actual path
# Load the CSV file
training_data = load_dataset('csv', data_files=file_path, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# Model and tokenizer names
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
refined_model = "finetuned_10labels"
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token

In [6]:
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)
# Training Params
training_args = TrainingArguments(
  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=3,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir= "./results",

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

)


In [13]:
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data.select(range(10)),
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args= training_args 
)

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

[codecarbon INFO @ 11:16:25] [setup] RAM Tracking...
[codecarbon INFO @ 11:16:25] [setup] GPU Tracking...
[codecarbon INFO @ 11:16:25] No GPU found.
[codecarbon INFO @ 11:16:25] [setup] CPU Tracking...
[codecarbon WARNING @ 11:16:25] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 11:16:27] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1255U but we don't know it. Please contact us.
[codecarbon INFO @ 11:16:27] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 11:16:27] >>> Tracker's metadata:
[codecarbon INFO @ 11:16:27]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 11:16:27]   Python version: 3.9.18
[codecarbon INFO @ 11:16:27]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 11:16:27]   Available RAM : 63.692 GB
[codecarbon INFO @ 11:16:27]   CPU count: 12
[codecarbon INFO @ 11:16:27]   CPU model: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 11:16:27]   GPU count: None
[

In [14]:
fine_tuning.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\spurt/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[codecarbon INFO @ 11:19:49] Energy consumed for RAM : 0.000100 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:19:49] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:19:49] 0.000277 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:20:04] Energy consumed for RAM : 0.000199 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:20:04] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:20:04] 0.000554 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:20:19] Energy consumed for RAM : 0.000298 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:20:19] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W
[codecarbo

Step,Training Loss,Validation Loss


[codecarbon INFO @ 11:26:50] Energy consumed for RAM : 0.002883 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:26:50] Energy consumed for all CPUs : 0.005139 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:26:50] 0.008022 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:27:05] Energy consumed for RAM : 0.002983 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:27:05] Energy consumed for all CPUs : 0.005316 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:27:05] 0.008299 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:27:20] Energy consumed for RAM : 0.003082 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:27:20] Energy consumed for all CPUs : 0.005493 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:27:20] 0.008575 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:27:35] Energy consumed for RAM : 0.003182 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 11:27:35] Energy consumed for all C

TrainOutput(global_step=3, training_loss=1.9792035818099976, metrics={'train_runtime': 1302.0053, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.002, 'total_flos': 333102503485440.0, 'train_loss': 1.9792035818099976, 'epoch': 1.2})

In [15]:
# Save Model
fine_tuning.model.save_pretrained(refined_model)

In [18]:
fine_tuning.save_model(refined_model)

In [19]:

fine_tuning_20_label = SFTTrainer(
    model=base_model,
    train_dataset=training_data.select(range(20)),
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args= training_args 
)

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

[codecarbon INFO @ 11:59:00] [setup] RAM Tracking...
[codecarbon INFO @ 11:59:00] [setup] GPU Tracking...
[codecarbon INFO @ 11:59:00] No GPU found.
[codecarbon INFO @ 11:59:00] [setup] CPU Tracking...
[codecarbon WARNING @ 11:59:00] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 11:59:01] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1255U but we don't know it. Please contact us.
[codecarbon INFO @ 11:59:01] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 11:59:01] >>> Tracker's metadata:
[codecarbon INFO @ 11:59:01]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 11:59:01]   Python version: 3.9.18
[codecarbon INFO @ 11:59:01]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 11:59:01]   Available RAM : 63.692 GB
[codecarbon INFO @ 11:59:01]   CPU count: 12
[codecarbon INFO @ 11:59:01]   CPU model: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 11:59:01]   GPU count: None
[

In [20]:
fine_tuning_20_label.train()

[codecarbon INFO @ 12:00:43] Energy consumed for RAM : 0.000100 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:00:43] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:00:43] 0.000277 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:00:58] Energy consumed for RAM : 0.000199 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:00:58] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:00:58] 0.000554 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:01:13] Energy consumed for RAM : 0.000298 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:01:13] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:01:13] 0.000830 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:01:28] Energy consumed for RAM : 0.000398 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:01:28] Energy consumed for all C

Step,Training Loss,Validation Loss


[codecarbon INFO @ 12:07:58] Energy consumed for RAM : 0.002983 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:07:58] Energy consumed for all CPUs : 0.005316 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:07:58] 0.008299 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:08:13] Energy consumed for RAM : 0.003083 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:08:13] Energy consumed for all CPUs : 0.005493 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:08:13] 0.008576 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:08:28] Energy consumed for RAM : 0.003182 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:08:28] Energy consumed for all CPUs : 0.005670 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:08:28] 0.008853 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:08:43] Energy consumed for RAM : 0.003282 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:08:43] Energy consumed for all C

TrainOutput(global_step=3, training_loss=1.8060898780822754, metrics={'train_runtime': 1290.3721, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.002, 'total_flos': 389432806563840.0, 'train_loss': 1.8060898780822754, 'epoch': 0.6})

In [21]:
refined_model_2 = "finetuned_20labels"

fine_tuning_20_label.save_model(refined_model_2)

In [23]:

fine_tuning_30_label = SFTTrainer(
    model=base_model,
    train_dataset=training_data.select(range(30)),
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args= training_args 
)

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

[codecarbon INFO @ 12:24:27] [setup] RAM Tracking...
[codecarbon INFO @ 12:24:27] [setup] GPU Tracking...
[codecarbon INFO @ 12:24:27] No GPU found.
[codecarbon INFO @ 12:24:27] [setup] CPU Tracking...
[codecarbon WARNING @ 12:24:27] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:24:28] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1255U but we don't know it. Please contact us.
[codecarbon INFO @ 12:24:28] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 12:24:28] >>> Tracker's metadata:
[codecarbon INFO @ 12:24:28]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 12:24:28]   Python version: 3.9.18
[codecarbon INFO @ 12:24:28]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 12:24:28]   Available RAM : 63.692 GB
[codecarbon INFO @ 12:24:28]   CPU count: 12
[codecarbon INFO @ 12:24:28]   CPU model: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 12:24:28]   GPU count: None
[

In [24]:
fine_tuning_30_label.train()

[codecarbon INFO @ 12:25:52] Energy consumed for RAM : 0.000100 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:25:52] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:25:52] 0.000277 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:26:07] Energy consumed for RAM : 0.000199 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:26:07] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:26:07] 0.000554 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:26:22] Energy consumed for RAM : 0.000299 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:26:22] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:26:22] 0.000831 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:26:37] Energy consumed for RAM : 0.000398 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:26:37] Energy consumed for all C

Step,Training Loss,Validation Loss


[codecarbon INFO @ 12:31:37] Energy consumed for RAM : 0.002387 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:31:37] Energy consumed for all CPUs : 0.004254 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:31:37] 0.006641 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:31:52] Energy consumed for RAM : 0.002487 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:31:52] Energy consumed for all CPUs : 0.004431 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:31:52] 0.006917 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:32:07] Energy consumed for RAM : 0.002586 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:32:07] Energy consumed for all CPUs : 0.004608 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:32:07] 0.007194 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:32:22] Energy consumed for RAM : 0.002685 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 12:32:22] Energy consumed for all C

TrainOutput(global_step=3, training_loss=1.8415309588114421, metrics={'train_runtime': 1387.09, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.002, 'total_flos': 377214684487680.0, 'train_loss': 1.8415309588114421, 'epoch': 0.4})

In [25]:
refined_model_3 = "finetuned_30labels"

fine_tuning_30_label.save_model(refined_model_3)

In [26]:

fine_tuning_40_label = SFTTrainer(
    model=base_model,
    train_dataset=training_data.select(range(40)),
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args= training_args 
)

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

[codecarbon INFO @ 13:07:44] [setup] RAM Tracking...
[codecarbon INFO @ 13:07:44] [setup] GPU Tracking...
[codecarbon INFO @ 13:07:44] No GPU found.
[codecarbon INFO @ 13:07:44] [setup] CPU Tracking...
[codecarbon WARNING @ 13:07:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:07:46] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1255U but we don't know it. Please contact us.
[codecarbon INFO @ 13:07:46] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 13:07:46] >>> Tracker's metadata:
[codecarbon INFO @ 13:07:46]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:07:46]   Python version: 3.9.18
[codecarbon INFO @ 13:07:46]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 13:07:46]   Available RAM : 63.692 GB
[codecarbon INFO @ 13:07:46]   CPU count: 12
[codecarbon INFO @ 13:07:46]   CPU model: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 13:07:46]   GPU count: None
[

In [27]:
fine_tuning_40_label.train()

[codecarbon INFO @ 13:08:33] Energy consumed for RAM : 0.000100 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:08:33] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:08:33] 0.000277 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:08:48] Energy consumed for RAM : 0.000199 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:08:48] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:08:48] 0.000554 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:09:03] Energy consumed for RAM : 0.000298 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:09:03] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:09:03] 0.000830 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:09:18] Energy consumed for RAM : 0.000398 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:09:18] Energy consumed for all C

Step,Training Loss,Validation Loss


[codecarbon INFO @ 13:16:03] Energy consumed for RAM : 0.003082 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:16:03] Energy consumed for all CPUs : 0.005492 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:16:03] 0.008574 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:16:18] Energy consumed for RAM : 0.003181 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:16:18] Energy consumed for all CPUs : 0.005669 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:16:18] 0.008851 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:16:33] Energy consumed for RAM : 0.003281 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:16:33] Energy consumed for all CPUs : 0.005847 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:16:33] 0.009128 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:16:48] Energy consumed for RAM : 0.003380 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:16:48] Energy consumed for all C

TrainOutput(global_step=3, training_loss=1.7681265274683635, metrics={'train_runtime': 1271.8388, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.002, 'total_flos': 376381630709760.0, 'train_loss': 1.7681265274683635, 'epoch': 0.3})

In [28]:
refined_model_4 = "finetuned_40labels"

fine_tuning_40_label.save_model(refined_model_4)

In [29]:

fine_tuning_50_label = SFTTrainer(
    model=base_model,
    train_dataset=training_data.select(range(50)),
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args= training_args 
)

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

[codecarbon INFO @ 13:41:33] [setup] RAM Tracking...
[codecarbon INFO @ 13:41:33] [setup] GPU Tracking...
[codecarbon INFO @ 13:41:33] No GPU found.
[codecarbon INFO @ 13:41:33] [setup] CPU Tracking...
[codecarbon WARNING @ 13:41:33] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:41:36] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1255U but we don't know it. Please contact us.
[codecarbon INFO @ 13:41:36] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 13:41:36] >>> Tracker's metadata:
[codecarbon INFO @ 13:41:36]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:41:36]   Python version: 3.9.18
[codecarbon INFO @ 13:41:36]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 13:41:36]   Available RAM : 63.692 GB
[codecarbon INFO @ 13:41:36]   CPU count: 12
[codecarbon INFO @ 13:41:36]   CPU model: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 13:41:36]   GPU count: None
[

In [30]:
fine_tuning_50_label.train()

[codecarbon INFO @ 13:42:55] Energy consumed for RAM : 0.000100 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:42:55] Energy consumed for all CPUs : 0.000178 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:42:55] 0.000278 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:43:10] Energy consumed for RAM : 0.000199 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:43:10] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:43:10] 0.000554 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:43:25] Energy consumed for RAM : 0.000299 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:43:25] Energy consumed for all CPUs : 0.000533 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:43:25] 0.000831 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:43:40] Energy consumed for RAM : 0.000398 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:43:40] Energy consumed for all C

Step,Training Loss,Validation Loss


[codecarbon INFO @ 13:59:44] Energy consumed for RAM : 0.006777 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:59:44] Energy consumed for all CPUs : 0.012084 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:59:44] 0.018860 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:59:59] Energy consumed for RAM : 0.006876 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:59:59] Energy consumed for all CPUs : 0.012261 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:59:59] 0.019137 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:00:14] Energy consumed for RAM : 0.006976 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:00:14] Energy consumed for all CPUs : 0.012439 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:00:14] 0.019415 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:00:29] Energy consumed for RAM : 0.007075 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:00:29] Energy consumed for all C

TrainOutput(global_step=3, training_loss=1.8592156569163005, metrics={'train_runtime': 4124.3749, 'train_samples_per_second': 0.003, 'train_steps_per_second': 0.001, 'total_flos': 377492369080320.0, 'train_loss': 1.8592156569163005, 'epoch': 0.24})

In [31]:
refined_model_5 = "finetuned_50labels"

fine_tuning_30_label.save_model(refined_model_5)